# NVIDIA Meeting Recorder Agent — Pro & Refactored

This notebook walks you through building an **AI Meeting Recorder** that:
- Reads meetings from **Google Calendar**
- Spins up a recording bot, monitors it, and retrieves assets
- Emails video, transcript & summary

It leverages **xpander.ai** for orchestration and **NVIDIA NIM** for the LLM.

## 1  Prerequisites
1. **Python ≥ 3.10**
2. **Node.js** (for the xpander CLI)
3. A `.env` file containing:
```env
XPANDER_API_KEY=...
NVIDIA_NIM_API_KEY=...
XPANDER_AGENT_ID=...
```

## 2  Install & Login
```bash
# Install xpander CLI (skip if already installed)
npm install -g xpander-cli
xpander login    # copy your XPANDER_API_KEY to your .env file
```

## 3  Set Up and Build the AI Agent

Skip manual configuration: use the pre‑built xpander template below to create the agent with all necessary tools wired up.

<div style="text-align: center;">
  <a href="https://app.xpander.ai/templates/48039a71-c99c-4691-8b66-a6faca3ccbe4" target="_blank">
    <button style="background-color:#6741d9;color:white;padding:10px 20px;font-size:14px;border:none;border-radius:8px;cursor:pointer;">
      Build NVIDIA Meeting Recorder Agent with xpander Template
    </button>
  </a>
</div>


Copy the Agent ID from the URL (https://app.xpander.ai/agents/{agent-id}) and add it to your .env file as XPANDER_AGENT_ID={agent-id}.

In [ ]:
import os, time, datetime as dt
from dotenv import load_dotenv
from xpander_sdk import XpanderClient, LLMProvider, Agent, Tokens, LLMTokens
from openai import OpenAI
import json

load_dotenv()

REQUIRED = ["XPANDER_API_KEY", "NVIDIA_NIM_API_KEY", "XPANDER_AGENT_ID"]
missing = [k for k in REQUIRED if os.getenv(k) is None]
if missing:
    raise EnvironmentError(f"Missing env vars: {', '.join(missing)}")

XPANDER_API_KEY    = os.getenv("XPANDER_API_KEY")
NVIDIA_NIM_API_KEY = os.getenv("NVIDIA_NIM_API_KEY")
AGENT_ID           = os.getenv("XPANDER_AGENT_ID")

print("✅ Environment ready — open your agent at",
      f"https://app.xpander.ai/agents/{AGENT_ID}")

## 4  MeetingAgent Class

In [ ]:

class MeetingAgent:
    """Wrapper around an xpander agent backed by NVIDIA NIM."""

    MODEL = "nvidia/llama-3.1-nemotron-ultra-253b-v1"

    def __init__(self, xpander_key: str, nim_key: str, agent_id: str):
        self.client      = XpanderClient(api_key=xpander_key)
        self.agent: Agent= self.client.agents.get(agent_id=agent_id)
        self.agent.select_llm_provider(llm_provider=LLMProvider.NVIDIA_NIM)
        self.llm         = OpenAI(base_url="https://integrate.api.nvidia.com/v1",
                                  api_key=nim_key)

    # ------------------------------------------------------------------
    def run(self, prompt: str, *, thread_id: str | None = None):
        """Run a single prompt; returns (text, thread_id)."""
        self.agent.add_task(input=prompt, thread_id=thread_id)
        
        current_time = dt.datetime.now().isoformat(sep=' ', timespec='seconds')
        
        # add current time for fresh threads
        if not thread_id:
            self.agent.add_messages([{
                "role": "system",
                "content": f"Current time: {current_time}"
            }])

        usage = Tokens(worker=LLMTokens())   # token accounting
        t_start = time.perf_counter()

        while not self.agent.is_finished():
            t_loop = time.perf_counter()
            res = self.llm.chat.completions.create(
                model       = self.MODEL,
                messages    = self.agent.messages,
                temperature = 0,
                tools       = self.agent.get_tools(),
                tool_choice = self.agent.tool_choice,
            )
            
            # count tokens
            u = res.usage
            usage.worker.completion_tokens += u.completion_tokens
            usage.worker.prompt_tokens     += u.prompt_tokens
            usage.worker.total_tokens      += u.total_tokens

            # report execution llm metrics
            self.agent.report_llm_usage(
                res.model_dump(),
                time.perf_counter() - t_loop
            )
            
            # add llm response to the agent's memory
            self.agent.add_messages(res.model_dump())

            # extract tool calls and run (cloud)
            tool_calls = self.agent.extract_tool_calls(res.model_dump())
            self.agent.run_tools(tool_calls)

        result = self.agent.retrieve_execution_result()
        self.agent.report_execution_metrics(
            llm_tokens=usage,
            ai_model=self.MODEL
        )
        duration = time.perf_counter() - t_start
        print(f"⏱ Finished in {duration:.1f}s — status: {result.status}")
        return result.result, result.memory_thread_id


## 5  Initialize Agent & Provide Instructions

In [ ]:

meeting_agent = MeetingAgent(XPANDER_API_KEY, NVIDIA_NIM_API_KEY, AGENT_ID)

TOOLS_JSON = json.dumps(meeting_agent.agent.get_tools())

# set the agent's instructions
meeting_agent.agent.instructions.general = f"""
Reasoning mode: ON
You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
To list user's calendar, use CalendarEventManagementGetCalendarEventsById
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of <TOOLCALL>[func_name1{{'params_name1': 'params_value1', 'params_name2': 'params_value2'...}}]</TOOLCALL>

You SHOULD NOT include any other text in the response.
Here is a list of functions in JSON format that you can invoke. 

<AVAILABLE_TOOLS>{TOOLS_JSON}</AVAILABLE_TOOLS>
"""

meeting_agent.agent.instructions.role = """
You are a helpful meeting recorder AI agent with access to Google Calendar and Meeting Recording tools.

"""

meeting_agent.agent.instructions.goal = """
Your goal is to help users answer their query, record their meetings, check the status of the recording tools, and get the assets of the recorded meeting using the recording bot id. However, please don't check the status of recorder unless explicitly requested by the user.

"""



## 6  Quick Demo

In [ ]:
# creates a thread and running the agentic loop
text, thread = meeting_agent.run('Hi! What can you do?')
print(text)

In [ ]:
# adding a message to the existing thread and running the agentic loop
meeting_agent.run('List my upcoming meetings on <date> and the three days after, including title, time and participants.', thread_id=thread)

In [ ]:
# adding another message to the existing thread and running the agentic loop
meeting_agent.run('Create a recorder for the onboarding meeting.', thread_id=thread)

In [ ]:
meeting_agent.run('Check the recorder status and give me the asset links if done.', thread_id=thread)

In [ ]:
meeting_agent.run('Email the video & transcript to <your_email@example.com>  with a 5‑bullet summary.', thread_id=thread)

## 7  Next Steps
* Add more calendar providers or messaging channels.
* Extend with custom tools (e.g. Slack notifications).
* Multi‑user auth & context isolation.

📚 See the [xpander docs](https://docs.xpander.ai) for deeper integrations.